# Calculate Circadian Effects
According to [https://pubmed.ncbi.nlm.nih.gov/8726347/](https://pubmed.ncbi.nlm.nih.gov/8726347/), circadian effects have impact on athletic performance. In order to provide detail to this effect, let us calculate the number of timezones traveled between games for each team.

In [14]:
import requests
import json
import pandas as pd

##### Read in game data

In [2]:
years = ['2019']
games = {}
for year in years:
    path = '../../nhl_overtime_eda/data/' + year + '_regSeasonPlays.json'
    print(path)
    with open(path) as f:
        games[year] = json.load(f)

../../nhl_overtime_eda/data/2019_regSeasonPlays.json


##### Extract data
Get game_id, home and away team, and date for each game

In [20]:
games_list = []

for season in games:
    for g_id in games[season]:
        try:
            games_list.append({
                'game_id': g_id,
                'home_team': games[season][g_id]['home_team'],
                'away_team': games[season][g_id]['away_team'],
                'date': games[season][g_id]['events'][0]['about']['dateTime']
            })
        except:
            pass # game posteponed or canceled (COVID-19 Season)

##### Convert to df, add features

In [55]:
df = pd.DataFrame(games_list)

# convert date string to datetime
df['date'] = pd.to_datetime(df['date'].str[:10]) 

# add location of game column
df['location'] = df['home_team']

# convert df from wide to long
df1 = df[['game_id', 'home_team', 'date', 'location']]
df1.columns = ['game_id', 'team', 'date', 'location']
df2 = df[['game_id', 'away_team', 'date', 'location']]
df2.columns = ['game_id', 'team', 'date', 'location']
df = pd.concat([df1, df2], ignore_index=True)

# merge timezones of team and location
df.sort_values(['team', 'date'], inplace=True)
df['location_prev'] = df.groupby(['team'])['location'].shift(1).fillna(df['location'])
# ...add code

df.head()

,game_id,team,date,location,location_prev
11,2019020012,Anaheim Ducks,2019-10-04,Anaheim Ducks,Anaheim Ducks
29,2019020030,Anaheim Ducks,2019-10-05,Anaheim Ducks,Anaheim Ducks
1121,2019020040,Anaheim Ducks,2019-10-08,Detroit Red Wings,Anaheim Ducks
1131,2019020050,Anaheim Ducks,2019-10-10,Pittsburgh Penguins,Detroit Red Wings
1140,2019020059,Anaheim Ducks,2019-10-11,Columbus Blue Jackets,Pittsburgh Penguins


pause and retrieve timezone data

In [49]:
# retrieve timezone data from nhl api
r = requests.get(url='https://statsapi.web.nhl.com/api/v1/teams')
d = r.json()

# extract data
team_tz = []
for team in d['teams']:
    team_tz.append({
        'team': team['name'],
        'timeZone': team['venue']['timeZone']['tz'],
        'offset': team['venue']['timeZone']['offset']
    })
    
# convert to df
df_tz = pd.DataFrame(team_tz)
df_tz.sort_values('team', inplace=True)

continue with calculating timezone canges

In [67]:
# get timezone/offset of current location
df_tz.columns = ['location', 'timeZone', 'offset']
df = df.sort_values('location').merge(df_tz, how='left', on='location')

# get timezone/offset of current location
df_tz.columns = ['location_prev', 'timeZone_prev', 'offset_prev']
df = df.sort_values('location_prev').merge(df_tz, how='left', on='location_prev')

# calculate change in offset
df = df.sort_values(['team', 'date'])
df['offset_diff'] = df['offset'] - df['offset_prev']


In [68]:
df[df['team'] == 'Carolina Hurricanes'].head(30)

,game_id,team,date,location,location_prev,timeZone,offset,timeZone_prev,offset_prev,offset_diff
342,2019020008,Carolina Hurricanes,2019-10-03,Carolina Hurricanes,Carolina Hurricanes,EDT,-4,EDT,-4,0
358,2019020023,Carolina Hurricanes,2019-10-05,Washington Capitals,Carolina Hurricanes,EDT,-4,EDT,-4,0
2087,2019020031,Carolina Hurricanes,2019-10-06,Carolina Hurricanes,Washington Capitals,EDT,-4,EDT,-4,0
397,2019020036,Carolina Hurricanes,2019-10-08,Florida Panthers,Carolina Hurricanes,EDT,-4,EDT,-4,0
858,2019020060,Carolina Hurricanes,2019-10-11,Carolina Hurricanes,Florida Panthers,EDT,-4,EDT,-4,0
344,2019020068,Carolina Hurricanes,2019-10-12,Carolina Hurricanes,Carolina Hurricanes,EDT,-4,EDT,-4,0
406,2019020091,Carolina Hurricanes,2019-10-16,Los Angeles Kings,Carolina Hurricanes,PDT,-7,EDT,-4,-3
908,2019020097,Carolina Hurricanes,2019-10-17,San Jose Sharks,Los Angeles Kings,PDT,-7,PDT,-7,0
1646,2019020112,Carolina Hurricanes,2019-10-18,Anaheim Ducks,San Jose Sharks,PDT,-7,PDT,-7,0
45,2019020148,Carolina Hurricanes,2019-10-24,Columbus Blue Jackets,Anaheim Ducks,EDT,-4,PDT,-7,3


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2164 entries, 0 to 2163
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   game_id   2164 non-null   object        
 1   team      2164 non-null   object        
 2   date      2164 non-null   datetime64[ns]
 3   location  2164 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 67.8+ KB


##### Save to file

In [82]:
result = df.to_json(orient="records")
parsed = json.loads(result)
path = '../data/teamCircadian.json'
with open(path, 'w') as f:
    json.dump(parsed, f, indent=2)

##### To re-open file

In [83]:
with open('../data/teamCircadian.json') as f:
    d = json.load(f)          # read file
df_tmp = pd.DataFrame(d)      # convert to df

In [84]:
df_tmp

,game_id,team,date,location,location_prev,timeZone,offset,timeZone_prev,offset_prev,offset_diff
0,2019020012,Anaheim Ducks,1570147200000,Anaheim Ducks,Anaheim Ducks,PDT,-7,PDT,-7,0
1,2019020030,Anaheim Ducks,1570233600000,Anaheim Ducks,Anaheim Ducks,PDT,-7,PDT,-7,0
2,2019020040,Anaheim Ducks,1570492800000,Detroit Red Wings,Anaheim Ducks,EDT,-4,PDT,-7,3
3,2019020050,Anaheim Ducks,1570665600000,Pittsburgh Penguins,Detroit Red Wings,EDT,-4,EDT,-4,0
4,2019020059,Anaheim Ducks,1570752000000,Columbus Blue Jackets,Pittsburgh Penguins,EDT,-4,EDT,-4,0
...,...,...,...,...,...,...,...,...,...,...
2159,2019021009,Winnipeg Jets,1583020800000,Edmonton Oilers,Winnipeg Jets,MDT,-6,CDT,-5,-1
2160,2019021024,Winnipeg Jets,1583280000000,Winnipeg Jets,Edmonton Oilers,CDT,-5,MDT,-6,1
2161,2019021045,Winnipeg Jets,1583539200000,Winnipeg Jets,Winnipeg Jets,CDT,-5,CDT,-5,0
2162,2019021067,Winnipeg Jets,1583712000000,Winnipeg Jets,Winnipeg Jets,CDT,-5,CDT,-5,0
